<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/CCAP_UNET_processing_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import shutil
import hashlib

In [3]:
# # OR Unzipping to a specific directory
!mkdir /content/CCAP

In [36]:
!mkdir '/content/Haleiwa2022'
!mkdir '/content/Haleiwa2022/Images'
!mkdir '/content/Haleiwa2022/Labels'

In [37]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/Haleiwa2022.zip -d /content/CCAP/

Archive:  /content/drive/MyDrive/data/wetland_project/processed_data/planet/Haleiwa2022.zip
   creating: /content/CCAP/content/Haleiwa2022/Images/
  inflating: /content/CCAP/content/Haleiwa2022/Images/1_1.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1_1025.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1_1537.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1_2049.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1_2553.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1_513.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1025_1.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1025_1025.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1025_1537.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1025_2049.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1025_2553.tif  
  inflating: /content/CCAP/content/Haleiwa2022/Images/1025_513.tif  
  inflating: /content/CCAP/content/Hale

Next two blocks of code are probably unnecessary but run them for a sanity check.

In [38]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Haleiwa2022/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Haleiwa2022/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

The number of files in the folder is: 30
The number of files in the folder is: 30


In [39]:
df = pd.read_csv('/content/CCAP/content/Haleiwa2022/Haleiwa2022.csv')

In [40]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [41]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent
10,1537,513,1025,1,3,592870.389353,594099.185750,2.389698e+06,2.390927e+06,15,0.000389
49,1537,1025,1025,513,8,592870.373073,594099.195167,2.388469e+06,2.389698e+06,13,0.033924
50,1537,1025,1025,513,8,592870.373073,594099.195167,2.388469e+06,2.389698e+06,14,0.226032
51,1537,1025,1025,513,8,592870.373073,594099.195167,2.388469e+06,2.389698e+06,15,0.023796
61,2049,1025,1537,513,9,594099.221412,595327.997108,2.388469e+06,2.389698e+06,15,0.010429
69,2353,1025,1841,513,10,594828.821189,596057.597323,2.388469e+06,2.389698e+06,15,0.015278
86,1025,1537,513,1025,12,591641.603972,592870.408640,2.387240e+06,2.388469e+06,14,0.009796
87,1025,1537,513,1025,12,591641.603972,592870.408640,2.387240e+06,2.388469e+06,15,0.064041
97,1537,1537,1025,1025,13,592870.406502,594099.220294,2.387240e+06,2.388469e+06,13,0.001465
98,1537,1537,1025,1025,13,592870.406502,594099.220294,2.387240e+06,2.388469e+06,14,0.003139


In [42]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [43]:
subset_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name
10,1537,513,1025,1,3,592870.389353,594099.185750,2.389698e+06,2.390927e+06,15,0.000389,1025_1.tif
49,1537,1025,1025,513,8,592870.373073,594099.195167,2.388469e+06,2.389698e+06,13,0.033924,1025_513.tif
50,1537,1025,1025,513,8,592870.373073,594099.195167,2.388469e+06,2.389698e+06,14,0.226032,1025_513.tif
51,1537,1025,1025,513,8,592870.373073,594099.195167,2.388469e+06,2.389698e+06,15,0.023796,1025_513.tif
61,2049,1025,1537,513,9,594099.221412,595327.997108,2.388469e+06,2.389698e+06,15,0.010429,1537_513.tif
69,2353,1025,1841,513,10,594828.821189,596057.597323,2.388469e+06,2.389698e+06,15,0.015278,1841_513.tif
86,1025,1537,513,1025,12,591641.603972,592870.408640,2.387240e+06,2.388469e+06,14,0.009796,513_1025.tif
87,1025,1537,513,1025,12,591641.603972,592870.408640,2.387240e+06,2.388469e+06,15,0.064041,513_1025.tif
97,1537,1537,1025,1025,13,592870.406502,594099.220294,2.387240e+06,2.388469e+06,13,0.001465,1025_1025.tif
98,1537,1537,1025,1025,13,592870.406502,594099.220294,2.387240e+06,2.388469e+06,14,0.003139,1025_1025.tif


In [44]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Haleiwa2022/Images'
destination_images = '/content/Haleiwa2022/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

Copied 1025_1.tif to /content/Haleiwa2022/Images
Copied 1025_513.tif to /content/Haleiwa2022/Images
Copied 1025_513.tif to /content/Haleiwa2022/Images
Copied 1025_513.tif to /content/Haleiwa2022/Images
Copied 1537_513.tif to /content/Haleiwa2022/Images
Copied 1841_513.tif to /content/Haleiwa2022/Images
Copied 513_1025.tif to /content/Haleiwa2022/Images
Copied 513_1025.tif to /content/Haleiwa2022/Images
Copied 1025_1025.tif to /content/Haleiwa2022/Images
Copied 1025_1025.tif to /content/Haleiwa2022/Images
Copied 1025_1025.tif to /content/Haleiwa2022/Images
Copied 1_1537.tif to /content/Haleiwa2022/Images
Copied 1_1537.tif to /content/Haleiwa2022/Images
Copied 1_1537.tif to /content/Haleiwa2022/Images
Copied 1_1537.tif to /content/Haleiwa2022/Images
Copied 513_1537.tif to /content/Haleiwa2022/Images
Copied 1025_1537.tif to /content/Haleiwa2022/Images
Copied 1537_1537.tif to /content/Haleiwa2022/Images
Copied 1_2049.tif to /content/Haleiwa2022/Images
Copied 1_2049.tif to /content/Haleiwa2

In [45]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Haleiwa2022/Labels'
destination_labels = '/content/Haleiwa2022/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

Copied 1025_1.tif to /content/Haleiwa2022/Labels
Copied 1025_513.tif to /content/Haleiwa2022/Labels
Copied 1025_513.tif to /content/Haleiwa2022/Labels
Copied 1025_513.tif to /content/Haleiwa2022/Labels
Copied 1537_513.tif to /content/Haleiwa2022/Labels
Copied 1841_513.tif to /content/Haleiwa2022/Labels
Copied 513_1025.tif to /content/Haleiwa2022/Labels
Copied 513_1025.tif to /content/Haleiwa2022/Labels
Copied 1025_1025.tif to /content/Haleiwa2022/Labels
Copied 1025_1025.tif to /content/Haleiwa2022/Labels
Copied 1025_1025.tif to /content/Haleiwa2022/Labels
Copied 1_1537.tif to /content/Haleiwa2022/Labels
Copied 1_1537.tif to /content/Haleiwa2022/Labels
Copied 1_1537.tif to /content/Haleiwa2022/Labels
Copied 1_1537.tif to /content/Haleiwa2022/Labels
Copied 513_1537.tif to /content/Haleiwa2022/Labels
Copied 1025_1537.tif to /content/Haleiwa2022/Labels
Copied 1537_1537.tif to /content/Haleiwa2022/Labels
Copied 1_2049.tif to /content/Haleiwa2022/Labels
Copied 1_2049.tif to /content/Haleiwa2

In [46]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Haleiwa2022/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Haleiwa2022/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

The number of files in the folder is: 16
The number of files in the folder is: 16


In [47]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Haleiwa2022_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

DataFrame exported to /content/Haleiwa2022_subset.csv


In [48]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Haleiwa2022'
zip_file_path = '/content/Haleiwa2022_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

ZIP file created: /content/Haleiwa2022_subset.zip


In [49]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/data/wetland_project/trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

ZIP file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata
CSV file moved to Google Drive: /content/drive/MyDrive/data/wetland_project/trainingdata


In [50]:
stopppsps"(W)

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-50-871ba97a253f>, line 1)

# Define next folder

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/data/wetland_project/processed_data/planet/Kahuku2021.zip -d /content/CCAP/

In [ ]:
!mkdir '/content/kahuku2021'
!mkdir '/content/kahuku2021/Images'
!mkdir '/content/kahuku2021/Labels'

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/kahuku2021/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/kahuku2021/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

In [ ]:
df = pd.read_csv('/content/CCAP/content/kahuku2021/kahuku2021.csv')

In [ ]:
df

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
subset_df

In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/kahuku2021/Images'
destination_images = '/content/kahuku2021/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/kahuku2021/Labels'
destination_labels = '/content/kahuku2021/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/kahuku2021/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/kahuku2021/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/kahuku2021_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/kahuku2021'
zip_file_path = '/content/kahuku2021_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')

# one more to do 7, 8, 9 together

In [ ]:
# Unzip the file and place the files to the newly cerated directory (CCAP/image_chips/ISLAND)
!unzip /content/drive/MyDrive/wetland_unet/UNET_Image_Chips/Hawaii_Island_Update/Export/Hawaii_2005_009.zip -d /content/CCAP/

In [ ]:
# original images and labels folder: how many files are there?
org_img = '/content/CCAP/content/Hawaii_2005_009/Images'
org_img_files = os.listdir(org_img)
org_img_count = len(org_img_files)
print(f'The number of files in the folder is: {org_img_count}')
org_label = '/content/CCAP/content/Hawaii_2005_009/Labels'
org_label_files = os.listdir(org_label)
org_label_count = len(org_label_files)
print(f'The number of files in the folder is: {org_label_count}')

In [ ]:
df = pd.read_csv('/content/CCAP/content/Hawaii_2005_009/Hawaii_2005_009.csv')

In [ ]:
subset_df = df[(df['label'] > 12) & (df['label'] < 19)]

In [ ]:
subset_df = subset_df.copy()  # Create a copy of the subset DataFrame to avoid modifying the original
subset_df['tif_name'] = subset_df['x_org'].astype(str) + "_" + subset_df['y_org'].astype(str) + ".tif"

In [ ]:
subset_df

In [ ]:
# Copy the useable images files to new folder
source_images = '/content/CCAP/content/Hawaii_2005_009/Images'
destination_images = '/content/Hawaii_2005_009/Images'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_images, tif_name)
    destination_path = os.path.join(destination_images, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_images}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_images}")

In [ ]:
# Copy the useable labels files to new folder
source_labels = '/content/CCAP/content/Hawaii_2005_009/Labels'
destination_labels = '/content/Hawaii_2005_009/Labels'
for tif_name in subset_df['tif_name']:
    source_path = os.path.join(source_labels, tif_name)
    destination_path = os.path.join(destination_labels, tif_name)
    try:
        shutil.copy(source_path, destination_path)
        print(f"Copied {tif_name} to {destination_labels}")
    except FileNotFoundError:
        print(f"File {tif_name} not found in {source_labels}")

In [ ]:
# NEW images and labels folder: how many files are there?
new_img = '/content/Hawaii_2005_009/Images'
new_img_files = os.listdir(new_img)
new_img_count = len(new_img_files)
print(f'The number of files in the folder is: {new_img_count}')
new_label = '/content/Hawaii_2005_009/Labels'
new_label_files = os.listdir(new_label)
new_label_count = len(new_label_files)
print(f'The number of files in the folder is: {new_label_count}')

In [ ]:
# Define the file path where you want to save the CSV file
csv_file_path = '/content/Hawaii_2005_009_subset.csv'
subset_df.to_csv(csv_file_path, index=False)
print(f"DataFrame exported to {csv_file_path}")

In [ ]:
# Define the path to the 'wetland_categories' folder
folder_to_zip = '/content/Hawaii_2005_009'
zip_file_path = '/content/Hawaii_2005_009_subset.zip'
shutil.make_archive(zip_file_path[:-4], 'zip', folder_to_zip)
print(f'ZIP file created: {zip_file_path}')

In [ ]:
# Define the destination folder in Google Drive
drive_folder = '/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata'
shutil.move(zip_file_path, drive_folder)
print(f'ZIP file moved to Google Drive: {drive_folder}')
shutil.move(csv_file_path, drive_folder)
print(f'CSV file moved to Google Drive: {drive_folder}')